## Load Data

In [ ]:
# install interpret if not already installed
try:
    import interpret
except ModuleNotFoundError:
    !pip install --quiet interpret numpy pandas scikit-learn

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

df = pd.read_csv(
    "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data",
    header=None)
df.columns = [
    "Age", "WorkClass", "fnlwgt", "Education", "EducationNum",
    "MaritalStatus", "Occupation", "Relationship", "Race", "Gender",
    "CapitalGain", "CapitalLoss", "HoursPerWeek", "NativeCountry", "Income"
]
X = df.iloc[:, :-1]
y = df.iloc[:, -1]

# Set feature_types manually instead of relying on auto-detect
feature_types = ['continuous', 'nominal', 'continuous', 'nominal',
    'continuous', 'nominal', 'nominal', 'nominal', 'nominal', 'nominal',
    'continuous', 'continuous', 'continuous', 'nominal']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

## Fit and compare DP-EBM vs. standard EBM

In [ ]:
from interpret.privacy import DPExplainableBoostingClassifier
from interpret.glassbox import ExplainableBoostingClassifier
from interpret import show

from sklearn.metrics import roc_auc_score

dpebm = DPExplainableBoostingClassifier(random_state=None, epsilon=1, delta=1e-6, feature_types=feature_types)
dpebm.fit(X_train, y_train)
dp_auc = roc_auc_score(y_test, dpebm.predict_proba(X_test)[:, 1])
print("DPEBM AUC: {:.3f}".format(dp_auc))

ebm = ExplainableBoostingClassifier()
ebm.fit(X_train, y_train)
ebm_auc = roc_auc_score(y_test, ebm.predict_proba(X_test)[:, 1])
print("EBM AUC: {:.3f}".format(ebm_auc))

## See differences in learned shape functions

In [ ]:
show(dpebm.explain_global(name='DP EBM'))

In [ ]:
show(ebm.explain_global(name='Standard EBM'))